In [3]:
import numpy as np
from collections import deque

# Set a random seed
np.random.seed(22)

# parameters
T = 10
b = 2
assets = ['stock1', 'stock2', 'bond', 'money']
mu_mean = {'stock1': 0.09, 'stock2': 0.07, 'bond': 0.04, 'money': 0.02}
sigma = {'stock1': 0.15, 'stock2': 0.11, 'bond': 0.05, 'money': 0.01}

asset_idx = {a: i for i, a in enumerate(assets)}

# correlation matrix
corr_matrix = np.array([
    [1.0, 0.4, 0.2, 0.1],
    [0.4, 1.0, 0.1, 0.1],
    [0.2, 0.1, 1.0, 0.3],
    [0.1, 0.1, 0.3, 1.0]
])
std_vec = np.array([sigma[a] for a in assets])
D = np.diag(std_vec)
Sigma = D @ corr_matrix @ D

# AR(1) parameters
phi = 0.3 * np.eye(len(assets))
mu_vec = np.array([mu_mean[a] for a in assets])

# Initialize tree structure
tree = {}
structure = []
mu = {}

counter = 0
node_map = {}
queue = deque()
root_key = (0,)
root_name = f"n{counter}"
node_map[root_key] = root_name
tree[root_name] = np.zeros(len(assets))
structure.append({"node": root_name, "parent": "none", "stage": 0, "prob": 1.0})
queue.append((root_key, root_name, 0, 1.0))
counter += 1

while queue:
    parent_key, parent_name, stage, prob = queue.popleft()
    if stage >= T:
        continue
    for i in range(b):
        child_key = parent_key + (i,)
        child_name = f"n{counter}"
        node_map[child_key] = child_name

        eps = np.random.multivariate_normal(np.zeros(len(assets)), Sigma)
        r_parent = tree[parent_name]
        r_child = mu_vec + phi @ (r_parent - mu_vec) + eps

        tree[child_name] = r_child
        structure.append({
            "node": child_name,
            "parent": parent_name,
            "stage": stage + 1,
            "prob": prob / b
        })
        queue.append((child_key, child_name, stage + 1, prob / b))
        counter += 1

# list of nodes 
nodes = [s["node"] for s in structure]

# compute mu 
for node in nodes:
    for i, a in enumerate(assets):
        mu[(a, node)] = tree[node][i]

# righting file .dat 

filename = "tree_AR1_10stages.dat"
with open(filename, 'w') as f:
    f.write("set A := " + " ".join(assets) + ";\n\n")
    f.write("set NODES :=\n")
    for t in range(T+1):
        stage_nodes = [s["node"] for s in structure if s["stage"] == t]
        f.write(" ".join(stage_nodes) + "\n")
    f.write(";\n\n")
    
    f.write(f"param T := {T};\n\n")
    
    f.write("param stage :=\n")
    for s in structure:
        f.write(f"{s['node']} {s['stage']} ")
    f.write(";\n\n")
    
    f.write("param parent :=\n")
    for s in structure:
        if s["parent"] != "none":
            f.write(f"{s['node']} {s['parent']} ")
    f.write(";\n\n")
    
    f.write("param prob :=\n")
    for s in structure:
        f.write(f"{s['node']} {s['prob']:.6f} ")
    f.write(";\n\n")
    
    f.write("param gamma := 2;\n\n")
    f.write("param w0 := 100000;\n\n")
    
    f.write("param mu:\n         " + " ".join(nodes) + " :=\n")
    for a in assets:
        f.write(f"{a}  ")
        for n in nodes:
            f.write(f"{mu[(a, n)]:.6f} ")
        f.write("\n")
    f.write(";\n")

print(f" File '{filename}' succesfully created with {len(nodes)} nodes on T={T}.")


 File 'tree_AR1_10stages.dat' succesfully created with 2047 nodes on T=10.


In [2]:
# Print simulated returns for the first stages
print("\nSimulated returns for each node (in the first stages):\n")
for t in range(4):
    stage_nodes = [s["node"] for s in structure if s["stage"] == t]
    print(f"Stage {t}:")
    for n in stage_nodes:
        returns = ", ".join(f"{a}: {mu[(a, n)]:+.4f}" for a in assets)
        print(f"  Node {n}: {returns}")
    print()


Simulated returns for each node (in the first stages):

Stage 0:
  Node n0: stock1: +0.0000, stock2: +0.0000, bond: +0.0000, money: +0.0000

Stage 1:
  Node n1: stock1: +0.0201, stock2: +0.1802, bond: -0.0269, money: +0.0091
  Node n2: stock1: +0.0960, stock2: +0.1689, bond: -0.0131, money: +0.0018

Stage 2:
  Node n3: stock1: -0.0442, stock2: +0.1070, bond: +0.0105, money: +0.0139
  Node n4: stock1: +0.0113, stock2: -0.0024, bond: +0.0662, money: +0.0289
  Node n5: stock1: +0.0330, stock2: -0.0423, bond: -0.0549, money: -0.0053
  Node n6: stock1: -0.2443, stock2: +0.1052, bond: +0.0314, money: +0.0179

Stage 3:
  Node n7: stock1: -0.0742, stock2: -0.0454, bond: +0.0031, money: +0.0209
  Node n8: stock1: +0.2166, stock2: +0.1563, bond: -0.0773, money: -0.0115
  Node n9: stock1: +0.0932, stock2: +0.0726, bond: +0.0935, money: +0.0243
  Node n10: stock1: +0.0411, stock2: +0.0625, bond: +0.0506, money: +0.0126
  Node n11: stock1: -0.0430, stock2: +0.0484, bond: +0.0563, money: +0.0351
  